# Qwen3-4B Fine-Tuning on NVIDIA L4 (QLoRA via Unsloth Nightly)
This notebook adapts **Qwen3-4B** with the `jmazz/sys-scan-linux-synthetic` corpus, following the repository's reliability-first MLOps guidance. Assume execution on a managed NVIDIA L4 (or similar bf16-capable GPU) with nightly Unsloth kernels installed.

**Safety & reproducibility guardrails**:
- Export credentials such as `HF_TOKEN` and `WANDB_API_KEY` _before_ running Section 1; never hard-code secrets in the notebook.
- Outputs remain synthetic and defensive—do not repurpose the pipeline for offensive tooling or real telemetry.
- Maintain a known-good adapter or base checkpoint to guarantee rollback paths if experiments diverge.

## 1. Set Up Environment
Provision the nightly Unsloth toolchain, validate the GPU, and register deterministic defaults before continuing.

**Checklist**
1. Remove any previously installed Unsloth wheels (prevents API drift).
2. Install nightly `unsloth` and `unsloth_zoo` from source in editable mode.
3. Confirm the L4 exposes bf16 compute and at least 24 GiB VRAM.
4. Define shared constants (paths, seeds, dataset identifiers).
5. Restart the kernel after installation so CUDA kernels are refreshed.

In [ ]:
"""Install nightly Unsloth, verify GPU capability, and set reproducible defaults."""
from __future__ import annotations

import os
import random
import subprocess
import sys
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Iterable

import torch
import numpy as np

def _run_pip(args: Iterable[str], allow_failure: bool = False) -> None:
    """Execute pip safely, optionally tolerating failures (e.g., uninstall when missing)."""
    completed = subprocess.run([sys.executable, "-m", "pip", *args], check=False, text=True)
    if completed.returncode != 0 and not allow_failure:
        raise RuntimeError(
            f"pip {' '.join(args)} failed with code {completed.returncode}: {completed.stderr}"
        )

# Remove legacy wheels that might conflict with new nightly APIs.
for pkg in ("unsloth", "unsloth_zoo"):
    _run_pip(["uninstall", pkg, "-y"], allow_failure=True)

# Install Unsloth nightly (editable mode ensures consistent rebuilds if repo already cloned).
_run_pip([
    "install",
    "--no-deps",
    "--force-reinstall",
    "git+https://github.com/unslothai/unsloth.git@main#egg=unsloth",
])
_run_pip([
    "install",
    "--no-deps",
    "--force-reinstall",
    "git+https://github.com/unslothai/unsloth.git@main#subdirectory=unsloth_zoo&egg=unsloth_zoo",
])

def assert_l4_ready() -> None:
    if not torch.cuda.is_available():
        raise EnvironmentError("CUDA device not detected. Ensure the L4 is attached before continuing.")
    device = torch.device("cuda")
    name = torch.cuda.get_device_name(device)
    props = torch.cuda.get_device_properties(device)
    if "L4" not in name:
        raise EnvironmentError(f"Expected NVIDIA L4 GPU; detected {name} instead.")
    if not torch.cuda.is_bf16_supported():
        raise EnvironmentError("bf16 is required for this workflow. Update drivers/CUDA for L4 bf16 support.")
    total_gib = props.total_memory / 1024**3
    if total_gib < 24:
        raise EnvironmentError(
            f"Insufficient GPU memory: {total_gib:.1f} GiB detected; L4 instances expose ≥24 GiB.")
    print(
        f"GPU check passed — {name} with {total_gib:.1f} GiB, compute capability {props.major}.{props.minor}."
,
)

assert_l4_ready()

def seed_everything(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

@dataclass(frozen=True)
class ProjectPaths:
    project_root: Path = Path.cwd()
    dataset_id: str = "jmazz/sys-scan-linux-synthetic"
    dataset_revision: str = "main"
    log_dir: Path = project_root / "logs" / "qwen3_l4"
    output_dir: Path = project_root / "outputs" / "qwen3_l4"
    cache_dir: Path = project_root / "cache" / "hf-datasets"
    seed: int = 42
    run_stamp: str = datetime.utcnow().strftime("%Y%m%d_%H%M%S")

    def run_dir(self) -> Path:
        return self.output_dir / self.run_stamp

paths = ProjectPaths()

for directory in (paths.log_dir, paths.output_dir, paths.cache_dir, paths.run_dir()):
    directory.mkdir(parents=True, exist_ok=True)

seed_everything(paths.seed)
print(f"Deterministic seed set to {paths.seed}")
print(f"Artifacts will be saved to {paths.run_dir()}")

print("Restart the kernel now if any packages were (re)installed before proceeding.")

## 2. Load Dataset & Validate Schema
Use the curated sys-scan synthetic corpus from Hugging Face, enforce the ground-truth schema, and split deterministic shards for train/validation.